# Data Cleaning and Preparation

* During the course of doing data analysis and modeling, a significant amount of time
is spent on data preparation: loading, cleaning, transforming, and rearranging. Such
tasks are often reported to take up 80% or more of an analyst’s time. 
* Sometimes the
way that data is stored in files or databases is not in the right format for a particular
task. 
* Many researchers choose to do ad hoc processing of data from one form to
another using a general-purpose programming language, like Python, Perl, R, or Java,
or Unix text-processing tools like sed or awk. 
* Fortunately, pandas, along with the
built-in Python language features, provides you with a high-level, flexible, and fast set
of tools to enable you to manipulate data into the right form.


* In this chapter I discuss tools for missing data, duplicate data, string manipulation,
and some other analytical data transformations. 
* In the next chapter, I focus on combining and rearranging datasets in various ways.


In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 7.1  Handling Missing Data

* Missing data occurs commonly in many data analysis applications. 
* One of the goals
of pandas is to make working with missing data as painless as possible.
* For example,
all of the descriptive statistics on pandas objects exclude missing data by default.

* The way that missing data is represented in pandas objects is somewhat imperfect,
but it is functional for a lot of users. 
* For numeric data, pandas uses the floating-point
value **NaN** (Not a Number) to represent missing data. 
* We call this a *sentinel* value that
can be easily detected:

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [ ]:
string_data

In [ ]:
string_data.isnull()

* In pandas, we’ve adopted a convention used in the R programming language by referring to missing data as NA, which stands for *not available*. 
* In statistics applications,
NA data may either be data that does not exist or that exists but was not observed
(through problems with data collection, for example). 
* When cleaning up data for
analysis, it is often important to do analysis on the missing data itself to identify data
collection problems or potential biases in the data caused by missing data.

* The built-in Python **None** value is also treated as NA in object arrays:

In [ ]:
string_data[0] = None

In [ ]:
string_data

In [ ]:
string_data.isnull()

<img style="float: left;" src="pic/pic_7_1.png" width="600">

### Filtering Out Missing Data

* There are a few ways to filter out missing data. 
* While you always have the option to
do it by hand using **pandas.isnull** and boolean indexing, the **dropna** can be helpful.
* On a Series, it returns the Series with only the non-null data and index values:


In [ ]:
from numpy import nan as NA

In [ ]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [ ]:
data

In [ ]:
data.dropna()

This is equivalent to:

In [ ]:
data[data.notnull()]

In [ ]:
data.notnull()

* With DataFrame objects, things are a bit more complex. 
* You may want to drop rows
or columns that are all NA or only those containing any NAs. 
* **dropna** by default drops
any row containing a missing value:

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [ ]:
data

In [ ]:
cleaned = data.dropna()

In [ ]:
cleaned

Passing **how='all'** will only drop rows that are all NA:

In [ ]:
data.dropna(how='all')

To drop columns in the same way, pass axis=1:

In [ ]:
data[4] = NA

In [ ]:
data

In [ ]:
data.dropna(axis=1, how='all')

A related way to filter out DataFrame rows tends to concern time series data.   
Suppose
you want to keep only rows containing a certain number of observations.   
You can
indicate this with the **thresh** argument:

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))

In [ ]:
df

In [ ]:
df.iloc[:4, 1] = NA

In [ ]:
df

In [ ]:
df.iloc[:2, 2] = NA

In [ ]:
df

In [ ]:
df.dropna()

In [ ]:
df.dropna(thresh=2) # 2 개 이상의 non NA 값을 keep 한다.

### Filling In Missing Data

Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the “holes” in any number of ways.  
For most purposes, the **fillna** method is the workhorse function to use.   
Calling **fillna** with a constant replaces missing values with that value:

In [ ]:
df

In [ ]:
df.fillna(0)

Calling **fillna** with a dict, you can use a different fill value for each column:

In [ ]:
df.fillna({1: 0.5, 2: 0})

In [ ]:
df

**fillna** returns a new object, but you can modify the existing object in-place:

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df

The same interpolation methods available for reindexing can be used with **fillna**:

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

In [ ]:
df.fillna(method='ffill')

In [ ]:
df.fillna(method='ffill', limit=2)

With **fillna** you can do lots of other things with a little creativity.   
For example, you might pass the mean or median value of a Series:


In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

In [ ]:
data.mean()

In [ ]:
data.fillna(data.mean())

<img style="float: left;" src="pic/pic_7_2.png" width="500">

## Data Transformation

### Removing Duplicates

Duplicate rows may be found in a DataFrame for any number of reasons. 

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

The DataFrame method **duplicated** returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [ ]:
data.duplicated()

Relatedly, **drop_duplicates** returns a DataFrame where the duplicated array is False

In [ ]:
data.drop_duplicates()

Both of these methods by default consider all of the columns; alternatively, you can specify any subset of them to detect duplicates.   
Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column.

In [ ]:
data['v1'] = range(7)
data

In [ ]:
data.drop_duplicates(['k1'])

**duplicated** and **drop_duplicates** by default keep the first observed value combination. Passing keep='last' will return the last one.

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Function or Mapping

For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. 

Consider the following hypothetical data collected about various kinds of meat

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

Suppose you wanted to add a column indicating the type of animal that each food came from.   
Let’s write down a mapping of each distinct meat type to the kind of animal.


In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
meat_to_animal

The **map** method on a Series accepts a function or dict-like object containing a mapping, but here we have a small problem in that some of the meats are capitalized and others are not.   
Thus, we need to convert each value to lowercase using the **str.lower** Series method:


In [ ]:
data

In [ ]:
lowercased = data['food'].str.lower()
lowercased

In [ ]:
data['animal'] = lowercased.map(meat_to_animal)
data

We could also have passed a function that does all the work.

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### Replacing Values

Filling in missing data with the fillna method is a special case of more general value replacement.  
As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so.   
Let’s consider this Series

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

The -999 values might be sentinel values for missing data.   
To replace these with NA values that pandas understands, we can use replace, producing a new Series (unless you pass inplace=True).

In [ ]:
data.replace(-999, np.nan)

If you want to replace multiple values at once, you instead pass a list and then the substitute value.

In [ ]:
data.replace([-999, -1000], np.nan)

To use a different replacement for each value, pass a list of substitutes.

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

The argument passed can also be a dict.

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes

* Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.   
* You can also modify the axes in-place without creating a new data structure.   

Here’s a simple example:

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
data

Like a Series, the axis indexes have a map method:

In [ ]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

You can assign to index, modifying the DataFrame in-place:


In [ ]:
data.index = data.index.map(transform)
data

If you want to create a transformed version of a dataset without modifying the original, a useful method is *rename*:

In [ ]:
data.rename(index=str.title, columns=str.upper)

**rename** can be used in conjunction with a dict-like object providing new values for a subset of the axis labels.

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'pikachu'})

rename saves you from the chore of copying the DataFrame manually and assigning to its index and columns attributes. Should you wish to modify a dataset in-place, pass inplace=True:


In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

Continuous data is often discretized or otherwise separated into “bins” for analysis.   
Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

ages

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older.  
To do so, you have to use **cut**, a function in pandas.

In [ ]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)

In [ ]:
cats

The object pandas returns is a special Categorical object.   
The output you see describes the bins computed by pandas.cut.  
You can treat it like an array of strings indicating the bin name; internally it contains a categories array specifying the distinct category names along with a labeling for the ages data in the codes attribute.


In [ ]:
cats.codes

In [ ]:
cats.categories

In [ ]:
pd.value_counts(cats)

Note that **pd.value_counts**(cats) are the bin counts for the result of **pandas.cut**. 

Consistent with mathematical notation for intervals, a parenthesis means that the side is open, while the square bracket means it is closed (inclusive).  
You can change which side is closed by passing **right=False**:


In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

You can also pass your own bin names by passing a list or array to the **labels** option.

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [ ]:
pd.cut(ages, bins, labels=group_names)

If you pass an integer number of bins to **cut** instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data.   
Consider the case of some uniformly distributed data chopped into fourths:


In [ ]:
data = np.random.rand(20)
data

In [ ]:
data.min()

In [ ]:
data.max()

In [ ]:
pd.cut(data, 4, precision=2)

In [ ]:
data=np.arange(10)
print(data)
pd.cut(data, 2)

The **precision=2** option limits the decimal precision to two digits. 

A closely related function, **qcut**, bins the data based on sample quantiles.  
Depending on the distribution of the data, using **cut** will not usually result in each bin having the same number of data points.   
Since **qcut** uses sample quantiles instead, by definition you will obtain roughly equal-size bins:


In [ ]:
data = np.random.randn(1000)  # standard normal distribution

In [ ]:
cats = pd.qcut(data, 4)  # Cut into quartiles

In [ ]:
cats

In [ ]:
pd.value_counts(cats)

Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive).

In [ ]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

In [ ]:
cats=pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
pd.value_counts(cats)

### Detecting and Filtering Outliers

Filtering or transforming outliers is largely a matter of applying array operations.   
Consider a DataFrame with some normally distributed data:

In [ ]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

In [ ]:
data

Suppose you wanted to find values in one of the columns exceeding 3 in absolute value.

In [ ]:
col = data[2]
col[np.abs(col) > 3]

To select all rows having a value exceeding 3 or –3, you can use the any method on a boolean DataFrame.

In [ ]:
data[(np.abs(data) > 3).any(axis=1)] # axis=1 은 row 중에서 3이상인것이 있으면 그 row는 True

Values can be set based on these criteria. Here is code to cap values outside the interval –3 to 3.

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [ ]:
data

In [ ]:
data.iloc[41]  # 3 셀 앞의 결과에 나온 숫자 입력

In [ ]:
data.iloc[60]  # 3 셀 앞의 결과에 나온 숫자 입력

In [ ]:
data.describe()

The statement **np.sign(data)** produces 1 and –1 values based on whether the values in data are positive or negative:


In [ ]:
np.sign(data).head()

In [ ]:
data.head()

### Permutation and Random Sampling

Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the numpy.random.permutation function.   
Calling permutation with the length of the axis you want to permute produces an array of integers indicating the new ordering.


In [ ]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

In [ ]:
sampler = np.random.permutation(5)
sampler

That array can then be used in iloc-based indexing or the equivalent **take** function.

In [ ]:
df

In [ ]:
df.take(sampler)

To select a random subset without replacement, you can use the **sample** method on Series and DataFrame:

In [ ]:
df.sample(n=3)

To generate a sample with replacement (to allow repeat choices), pass **replace=True** to sample.

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables

* Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. 
* If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s.
* pandas has a get_dummies function for doing this, though devising one yourself is not difficult. 

Let’s return to an earlier example DataFrame

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})

In [ ]:
df

In [ ]:
pd.get_dummies(df['key'])

In some cases, you may want to add a prefix to the columns in the indicator DataFrame, which can then be merged with the other data.   
**get_dummies** has a prefix argument for doing this:


In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [ ]:
dummies

In [ ]:
df_with_dummy = df[['data1']].join(dummies)

In [ ]:
df_with_dummy

If a row in a DataFrame belongs to multiple categories, things are a bit more complicated.   
Let’s look at the MovieLens 1M dataset, which is investigated in more detail in Chapter 14:


## 생략가능 (7.3 으로 Go!!)

In [ ]:
mnames = ['movie_id', 'title', 'genres']

In [ ]:
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)

In [ ]:
movies[:10]

Adding indicator variables for each genre requires a little bit of wrangling.  
First, we extract the list of unique genres in the dataset:


In [ ]:
movies.iloc[0,2].split('|')

In [ ]:
movies.genres[0].split('|')

In [ ]:
all_genres = []

In [ ]:
for x in movies.genres:
    all_genres.extend(x.split('|'))

In [ ]:
genres = pd.unique(all_genres)

In [ ]:
genres

One way to construct the indicator DataFrame is to start with a DataFrame of all zeros:


In [ ]:
zero_matrix = np.zeros((len(movies), len(genres)))

In [ ]:
zero_matrix

In [ ]:
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [ ]:
dummies

Now, iterate through each movie and set entries in each row of dummies to 1.  
To do this, we use the **dummies.columns** to compute the column indices for each genre:


In [ ]:
gen = movies.genres[0]

In [ ]:
gen.split('|')

In [ ]:
dummies.columns.get_indexer(gen.split('|'))

Then, we can use **.iloc** to set values based on these indices:


In [ ]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [ ]:
dummies

Then, as before, you can combine this with movies.

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

In [ ]:
movies_windic.iloc[0]

In [ ]:
movies_windic

<img style="float: left;" src="pic/pic_0_2.png">

For much larger data, this method of constructing indicator variables with multiple membership is not especially speedy.   
It would be better to write a lower-level function that writes directly to a NumPy array, and then wrap the result in a DataFrame.

A useful recipe for statistical applications is to combine get_dummies with a discretization function like cut:

In [ ]:
np.random.seed(12345)
values = np.random.rand(10)
values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [ ]:
pd.get_dummies(pd.cut(values, bins))

We will look again at **pandas.get_dummies** later in the book. 

## 7.3  String Manipulation

Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing.   
Most text operations are made simple with the string object’s built-in methods.  
For more complex pattern matching and text manipulations, regular expressions may be needed.  
pandas adds to the mix by enabling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data. 

### String Object Methods

In many string munging and scripting applications, built-in string methods are sufficient.   
As an example, a comma-separated string can be broken into pieces with **split**.

In [ ]:
val = 'a,b,  guido'
val.split(',')

split is often combined with **strip** to trim whitespace (including line breaks).

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

These substrings could be concatenated together with a two-colon delimiter using addition.

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a list or tuple to the **join** method on the string '::'.

In [ ]:
'::'.join(pieces)

Other methods are concerned with locating substrings. Using Python’s in keyword is the best way to detect a substring, though **index** and **find** can also be used.

In [ ]:
val

In [ ]:
'guido' in val

In [ ]:
val.index(',')

In [ ]:
val.find(',')

In [ ]:
val.find(':')

Note the difference between find and index is that index raises an exception if the string isn’t found (versus returning –1).

In [ ]:
val.index(':')

 **count** returns the number of occurrences of a particular substring.

In [ ]:
val.count(',')

In [ ]:
val.count(':')

**replace** will substitute occurrences of one pattern for another.   
It is commonly used to delete patterns, too, by passing an empty string

In [ ]:
val.replace(',', '::')

In [ ]:
val.replace(',', '')

See Table 7-3 for a listing of some of Python’s string methods.   
Regular expressions can also be used with many of these operations, as you’ll see.


<img style="float: left;" src="pic/pic_7_3.png" width="600">

## 이번 챕터 끝 (다음 주제는 제대로 살펴보려면 시간부족!!)

### Regular Expressions

  
#### Regulare expression 을 제대로 학습하려면 양이 아주 많다.

* *Regular* expressions provide a flexible way to search or match (often more complex) string patterns in text.   
* A single expression, commonly called a *regex*, is a string formed according to the regular expression language.   
* Python’s built-in re module is responsible for applying regular expressions to strings. 


<img style="float: left;" src="pic/pic_0_2.png">

The art of writing regular expressions could be a chapter of its own and thus is outside the book’s scope.   
There are many excellent tutorials and references available on the internet and in other books.


The **re** module functions fall into three categories: pattern matching, substitution, and splitting.   
Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes.

Suppose we wanted to split a string with a variable number of whitespace characters (tabs, spaces, and newlines).   
The regex describing **one or more whitespace characters** is **\s+**:


In [ ]:
import re

In [ ]:
text = "foo    bar\t baz  \tqux"

In [ ]:
text

In [ ]:
re.split('\s+', text)

When you call re.split('\s+', text), the regular expression is first *compiled*, and then its **split** method is called on the passed text.   
You can compile the regex yourself with **re.compile**, forming a reusable regex object

In [ ]:
regex = re.compile('\s+')
regex.split(text)

If, instead, you wanted to get a list of all patterns matching the regex, you can use the findall method.

In [ ]:
regex.findall(text)

<img style="float: left;" src="pic/pic_0_2.png">

To avoid unwanted escaping with \ in a regular expression, use raw string literals like r'C:\x' instead of the equivalent 'C:\\x'.

Creating a regex object with **re.compile** is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles. 

**match** and **search** are closely related to **findall**.   
While **findall** returns all matches in a string, **search** returns only the first match.   
More rigidly, **match** only matches at the beginning of the string.   
As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:


In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

Using **findall** on the text produces a list of the email addresses.

In [ ]:
regex.findall(text)

**search** returns a special match object for the first email address in the text.   
For the preceding regex, the match object can only tell us the start and end position of the pattern in the string.

In [ ]:
m = regex.search(text)

In [ ]:
m

In [ ]:
text[m.start():m.end()]

Do not confuse **re.search()** with **re.match()**.   
Both functions do exactly the same, with the important distinction that **re.search()** will attempt the pattern throughout the string, until it finds a match.   
**re.match()** on the other hand, only attempts the pattern at the very start of the string. 

In [ ]:
print(regex.match(text))

In [ ]:
text="steve@gmail.com"
print(regex.match(text))

Relatedly, *sub* will return a new string with occurrences of the pattern replaced by the a new string

In [ ]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [ ]:
print(regex.sub('REDACTED', text))

Suppose you wanted to find email addresses and simultaneously segment each address into its three components: username, domain name, and domain suffix.  
To do this, put parentheses around the parts of the pattern to segment.


In [ ]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this modified regex returns a tuple of the pattern components with its **groups** method.

In [ ]:
m = regex.match('wesm@bright.net')
m.groups()

**findall** returns a list of tuples when the pattern has groups.

In [ ]:
regex.findall(text)

**sub** also has access to groups in each match using special symbols like \1 and \2.   
The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:


In [ ]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

There is much more to regular expressions in Python, most of which is outside the book’s scope. Table 7-4 provides a brief summary.

<img style="float: left;" src="pic/pic_7_4.png" width="600">

### Vectorized String Functions in pandas

Cleaning up a messy dataset for analysis often requires a lot of string munging and regularization.   
To complicate matters, a column containing strings will sometimes have missing data.

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

In [ ]:
data.isnull()

You can apply string and regular expression methods can be applied (passing a **lambda** or other function) to each value using **data.map**, but it will fail on the NA (null) values.   
To cope with this, Series has array-oriented methods for string operations that skip NA values.   
These are accessed through Series’s **str** attribute; for example, we could check whether each email address has 'gmail' in it with **str.contains**.


In [ ]:
data.str.contains('gmail')

Regular expressions can be used, too, along with any re options like IGNORECASE.

In [ ]:
pattern

In [ ]:
data.str.findall(pattern, flags=re.IGNORECASE)

There are a couple of ways to do vectorized element retrieval.   
Either use **str.get** or index into the **str** attribute

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

To access elements in the embedded lists, we can pass an index to either of these functions.

In [ ]:
matches.str.get(0)

In [ ]:
type(matches)

In [ ]:
type(matches.str)

In [ ]:
type(matches.astype(str))

In [ ]:
matches

In [ ]:
type(matches.astype(str).str)

In [ ]:
matches.astype(str).str.get(0)

In [ ]:
matches.astype(str).str[0]

In [ ]:
matches.astype(str)

In [ ]:
matches.str[0]

In [ ]:
matches=data.str.findall(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(0)

In [ ]:
matches.str[0]

You can similarly slice strings using this syntax.

In [ ]:
data

In [ ]:
data.str[:5]

<img style="float: left;" src="pic/pic_7_5.png" width="600">

<img style="float: left;" src="pic/pic_7_6.png" width="600">